### **Develop a GAN to Generate Small Photos**

This tutorial includes the following steps:

    * Obtain CIFAR-10 Small Object Photograph Dataset
    * Define and Train the Discriminator Model
    * Define and Use the Generator Model
    * Train the Generator Model
    * Evaluate GAN Model Performance
    * Use the Final Generator Model to Generate Images


#### **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# change the current dictory to the folder where you save the .h5 files
# For example, 
#os.chdir('/content/drive/My Drive/Colab Notebooks/GANs')
os.chdir('/content/drive/My Drive/') 


##### [CIFAR-10 ](https://www.cs.toronto.edu/~kriz/cifar.html)dataset was developed at the CIFAR institute, which is comprised of 60,000 32×32 pixel color photographs of objects from 10 classes, such as frogs, birds, cats, ships, airplanes, etc.

The cell below loads the dataset and summarizes the shape of the loaded dataset.

In [ ]:
# example of loading and plotting the cifar10 dataset
from keras.datasets.cifar10 import load_data
import matplotlib.pyplot as plt

# load the images into memory
(trainX, trainy), (testX, testy) = load_data()

# summarize the shape of the dataset
print('Train', trainX.shape, trainy.shape)
print('Test', testX.shape, testy.shape)

We can see that there are 50K examples in the training set and 10K in the test set and that each image is a square of 32 by 32 pixels.

The images are color with the object centered in the middle of the frame.  We can plot some of the images from the training dataset with the matplotlib library using the imshow() function.

below we plot the last 25 images from the training dataset in a 5 by 5 montage.

In [ ]:
# plot images from the training dataset
fig = plt.figure(figsize=(8,8))
for i in range(-25, 0):
	# define subplot
	ax = fig.add_subplot(5,5, 26+i)
 
	# turn off axis
	ax.xaxis.set_visible(False)
	ax.yaxis.set_visible(False)
 
	# plot raw pixel data
	ax.imshow(trainX[i])
plt.show()

#### **First, we define the discriminator model.**
The model takes a sample image from our dataset as input and outputs a classification prediction as either real or fake. This is a binary classification problem.

    Inputs: Image with three color channel and 32×32 pixels in size.
    Outputs: Binary classification, likelihood the sample is real (or fake).

The `define_discriminator()` function below defines the discriminator model and parametrizes the size of the input image.

In [ ]:
# example of defining the discriminator model
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
from keras.models import load_model
 
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.3))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define model
model = define_discriminator()
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

We could start training this model now with real examples with a class label of one and randomly generate samples with a class label of zero.
We use the cifar.load_data() function to load the CIFAR-10 dataset and just use the input part of the training dataset as the real images. Scale the pixel values from the range of unsigned integers in [0,255] to the normalized range of [-1,1].

In [ ]:
# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

The model will be updated in batches. On training an epoch is defined as one passes through the entire training cohort.<br><br> 
Training via stochastic gradient descent requires that the training dataset be shuffled prior to each epoch. A simpler approach is to select random samples of images from the training dataset.<br>
The `generate_real_samples()` function below will take the training dataset as an argument and will select a random subsample of images; it will also return class labels for the sample, specifically a class label of 1, to indicate real images.




In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

Next, we need a source of fake images.<br>
The `generate_fake_samples()` function below implements this behavior and generates images of random pixel values and their associated class label of 0, for fake.

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

#### **Secondly, define and Use the Generator Model**
The generator model is responsible for creating new, fake, but plausible small photographs of objects.

It does this by taking a point from the latent space as input and outputting a square color image.

The latent space is an arbitrarily defined vector space of Gaussian-distributed values, e.g. 100 dimensions. It has no meaning, but by drawing points from this space randomly and providing them to the generator model during training, the generator model will assign meaning to the latent points and, in turn, the latent space, until, at the end of training, the latent vector space represents a compressed representation of the output space, CIFAR-10 images, that only the generator knows how to turn into plausible CIFAR-10 images.

    Inputs: Point in latent space, e.g. a 100-element vector of Gaussian random numbers.
    Outputs: Two-dimensional square color image (3 channels) of 32 x 32 pixels with pixel values in [-1,1].

we use the LeakyReLU with a default slope of 0.2, reported as a best practice when training GAN models.

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

The `define_generator()` function below implements this and defines the generator model.

**Note:** the generator model is not compiled and does not specify a loss function or optimization algorithm. This is because the generator is not trained directly. 

Now summarize the model to help better understand the input and output shapes.

In [ ]:
# define the size of the latent space
latent_dim = 100
# define the generator model
model = define_generator(latent_dim)
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

To generate new points in the latent space, We can call the `randn()` NumPy function for generating arrays of random numbers drawn from a standard Gaussian.

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

We can now define the `generate_fake_samples()` function by taking the generator model as an argument and use it to generate the desired number of samples by first calling the `generate_latent_points()` function to generate the required number of points in latent space as input to the model.

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

Running the example generates 25 examples of fake CIFAR-10 images and visualizes them on a single plot of 5 by 5 images.

As the model is not trained, the generated images are completely random pixel values in [-1, 1], rescaled to [0, 1]. As we might expect, the images look like a mess of gray.

In [ ]:
# generate samples
n_samples = 25
X, _ = generate_fake_samples(model, latent_dim, n_samples)
# scale pixel values from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the generated samples
fig = plt.figure(figsize=(8,8))
for i in range(n_samples):
	# define subplot
	ax = fig.add_subplot(5, 5, i+1)
 
	# turn off axis labels
	ax.xaxis.set_visible(False)
	ax.yaxis.set_visible(False)
 
	# plot single image
	ax.imshow(X[i])
 
# show the figure
plt.show()

#### **Train the Generator Model**
The weights in the generator model are updated based on the performance of the discriminator model.

When the discriminator is good at detecting fake samples, the generator is updated more, and when the discriminator model is relatively poor or confused when detecting fake samples, the generator model is updated less.

This defines the zero-sum or adversarial relationship between these two models.<br>
One of the simplest approaches is to create a new model that combines the generator and discriminator models.  Therefore, the generator receives as input random points in the latent space and generates samples that are fed into the discriminator model directly, classified, and the output of this larger model can be used to update the model weights of the generator.


Only the discriminator is concerned with distinguishing between real and fake examples, therefore the discriminator model can be trained in a standalone manner on examples of each, as we did in the section on the discriminator model above.

The generator model is only concerned with the discriminator’s performance on fake examples. Therefore, we will mark all of the layers in the discriminator as not trainable when it is part of the GAN model so that they can not be updated and overtrained on fake examples.

We can imagine that the discriminator will then classify the generated samples as not real (class 0) or a low probability of being real (0.3 or 0.5). The backpropagation process used to update the model weights will see this as a large error and will update the model weights (i.e. only the weights in the generator) to correct for this error, in turn making the generator better at generating good fake samples.

    Inputs: Point in latent space, e.g. a 100-element vector of Gaussian random numbers.
    Outputs: Binary classification, likelihood the sample is real (or fake).

The `define_gan()` function below takes as arguments the already-defined generator and discriminator models and creates the new, logical third model subsuming these two models. The weights in the discriminator are marked as not trainable, which only affects the weights as seen by the GAN model and not the standalone discriminator model.

The GAN model then uses the same binary cross entropy loss function as the discriminator and the efficient Adam version of stochastic gradient descent with the learning rate of 0.0002 and momentum of 0.5, recommended when training deep convolutional GANs.

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

The trainable property impacts the model after it is compiled. The discriminator model was compiled with trainable layers, therefore the model weights in those layers will be updated when the standalone model is updated via calls to the `train_on_batch()` function.

The discriminator model was then marked as not trainable, added to the GAN model, and compiled. In this model, the model weights of the discriminator model are not trainable and cannot be changed when the GAN model is updated via calls to the `train_on_batch()` function. This change in the trainable property does not impact the training of the standalone discriminator model.

We first update the discriminator model with real and fake samples, then update the generator via the composite model. <br>
A few things to note in this model training function.
First, the number of batches within an epoch is defined by how many times the batch size divides into the training dataset.  The discriminator model is updated twice per batch, once with real samples and once with fake samples, which is a best practice as opposed to combining the samples and performing a single update. Last, Finally, the loss is reported in each batch. <br>

It is critical to keep an eye on the loss over batches. The reason for this is that a crash in the discriminator loss indicates that the generator model has started generating rubbish examples that the discriminator can easily discriminate.<br>

Monitor the discriminator loss and expect it to hover around 0.5 to 0.8 per batch. The generator loss is less critical and may hover between 0.5 and 2 or higher. 


In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			if (j+1) % 30 == 0:
				print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
 

#### **Evaluate GAN Model Performance**
There are no objective ways to evaluate the performance of a GAN model in general, which means, we cannot calculate this objective error score for generated images.<br>

Instead, images must be subjectively evaluated for quality by a human operator. This means that we cannot know when to stop training without looking at examples of generated images. In turn, the adversarial nature of the training process means that the generator is changing after every batch, meaning that once “good enough” images can be generated, the subjective quality of the images may then begin to vary, improve, or even degrade with subsequent updates.

There are three ways to handle this complex training situation.

    Periodically evaluate the classification accuracy of the discriminator on real and fake images.
    Periodically generate many images and save them to file for subjective review.
    Periodically save the generator model.
All three of these actions can be performed at the same time for a given training epoch, such as every 10 training epochs. The result will be a saved generator model for which we have a way of subjectively assessing the quality of its output and objectively knowing how well the discriminator was fooled at the time the model was saved.

First, we can define a function called `summarize_performance()` that will summarize the performance of the discriminator model. It does this by retrieving a sample of real CIFAR-10 images, as well as generating the same number of fake CIFAR-10 images with the generator model, then evaluating the classification accuracy of the discriminator model on each sample, and reporting these scores.<br>

This function can be called from the `train()` function based on the current epoch number, such as every 10 epochs.

Next, we update the `summarize_performance()` function to both save the model and to create and save a plot generated examples. The generator model can be saved by calling the save() function on the generator model and providing a unique filename based on the training epoch number.

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the discriminator and generator model tile files
	d_filename = 'discriminator_model_%03d.h5' % (epoch+1)
	g_filename = 'generator_model_%03d.h5' % (epoch+1)
	d_model.save(d_filename)
	g_model.save(g_filename)

We develop a function to create a plot of the generated samples.

In [ ]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=5):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	fig = plt.figure(figsize=(8,8))
	# plot images
	for i in range(n * n):
		# define subplot
		ax = fig.add_subplot(n, n, i+1)
		# turn off axis
		ax.xaxis.set_visible(False)
		ax.yaxis.set_visible(False)
		# plot raw pixel data
		ax.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

The model performance is reported every batch, including the loss of both the discriminative (d) and generative (g) models.<br>

In this case, the loss remains stable over the course of training. The discriminator loss on the real and generated examples sits around 0.5, whereas the loss for the generator trained via the discriminator sits around 1.5 for much of the training process.

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10)


`However, training more than 100 epochs will take too long to complete it. Therefore, loading the models saved after training for 20 epochs, for instance, will save some time. `

##### **First, your task is to modify the `train()` function to load the saved `g_model` and `d_model` and train the models for another 10 epochs.**<br>  **Note:** `train()` keeps all the features <br>
**Hint:** use `load_model(filename)` to load a model


In [ ]:
### modification
# train the generator and discriminator
from keras.models import load_model

pass

##### **Second, use the saved models (list below) as the auguments of the modifed `train()` function for training:**<br>
'generator_model_020.h5',<br>
'discriminator_model_020.h5'


In [ ]:
# size of the latent space
latent_dim = 100
dataset = load_real_samples()
# train model
pass

##### **Third, use the saved models (list below) as the auguments of the modifed `train()` function for training:**<br>
'generator_model_050.h5',<br>
'discriminator_model_050.h5'<br>
**After that
use the saved models (list below) as the auguments of the modifed `train()` function for training:**<br>
'generator_model_100.h5',<br>
'discriminator_model_100.h5'<br>

In [ ]:
# size of the latent space
latent_dim = 100
dataset = load_real_samples()
# train model
pass
pass

#### **Use the Final Generator Model to Generate Images**
This involves first loading the model from file, then using it to generate images. The generation of each image requires a point in the latent space as input.

In [ ]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
import matplotlib.pyplot as plt

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# plot the generated images
def create_plot(examples, n):
  # plot images
  fig = plt.figure(figsize=(8,8))
  for i in range(n * n):
		# define subplot
    ax = fig.add_subplot(n, n, 1 + i)
		# turn off axis
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
		# plot raw pixel data
    ax.imshow(examples[i, :, :])
  plt.show()

# load model
model = load_model('generator_model_200.h5')
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
create_plot(X, 10)

#### **Further exploration**

* **Change Latent Space.** Update the example to use a larger or smaller latent space and compare the quality of the results and speed of training.
* **Batch Normalization.** Update the discriminator and/or the generator to make use of batch normalization, recommended for DCGAN models.
* **Label Smoothing.** Update the example to use one-sided label smoothing when training the discriminator, specifically change the target label of real examples from 1.0 to 0.9 and add random noise, then review the effects on image quality and speed of training.
* **Model Configuration.** Update the model configuration to use deeper or more shallow discriminator and/or generator models, perhaps experiment with the UpSampling2D layers in the generator.


